# 🚀 NASA Defect Prediction: PC2\n\n**Dataset:** PC2\n**Method:** Baseline RF → KAN Base → KAN + Attention\n**Goal:** F2 & Recall optimization (defect detection)\n\n**✅ Self-contained:** Run all cells in order, no dependencies!\n\n---

## 📦 Step 1: Setup & Imports

In [ ]:
# ============================================================================\n# COMPLETE SETUP - ALL IMPORTS & CONFIGS\n# ============================================================================\n\n# Mount Google Drive\ntry:\n    from google.colab import drive\n    drive.mount('/content/drive')\n    print('✅ Google Drive mounted!')\nexcept ImportError:\n    print('⚠️  Not on Colab - skipping mount')\n\n# Install packages\nimport sys\n!{sys.executable} -m pip install imbalanced-learn scipy scikit-learn torch matplotlib seaborn pandas numpy openpyxl -q\n\n# Imports\nimport os\nimport json\nimport warnings\nimport datetime\nimport numpy as np\nimport pandas as pd\nfrom scipy.io import arff\nfrom io import StringIO\n\nfrom sklearn.preprocessing import MinMaxScaler, LabelEncoder\nfrom sklearn.model_selection import train_test_split\nfrom sklearn.ensemble import RandomForestClassifier\nfrom sklearn.metrics import (\n    accuracy_score, precision_score, recall_score, f1_score,\n    fbeta_score, confusion_matrix, average_precision_score\n)\nfrom imblearn.over_sampling import SMOTE\n\nimport torch\nimport torch.nn as nn\nimport torch.nn.functional as F\nimport torch.optim as optim\nfrom torch.utils.data import TensorDataset, DataLoader\n\nwarnings.filterwarnings('ignore')\ndevice = torch.device('cpu')\n\n# Configuration\nDATASET_NAME = 'PC2'\nDATASET_PATH = '/content/drive/MyDrive/nasa-defect-gwo-kan/dataset'\nOUTPUT_DIR = f'./results_{DATASET_NAME}'\nSEED = 42\n\n# Set seeds\nnp.random.seed(SEED)\ntorch.manual_seed(SEED)\nos.makedirs(OUTPUT_DIR, exist_ok=True)\n\nRUN_ID = datetime.datetime.now().strftime('%Y%m%d_%H%M%S')\n\nprint(f'✅ Setup complete!')\nprint(f'📊 Dataset: PC2')\nprint(f'🖥️  Device: {device}')\nprint(f'📁 Output: {OUTPUT_DIR}')\n

## 🛠️ Step 2: Define All Functions & Models

In [ ]:
# ============================================================================\n# UTILITY FUNCTIONS\n# ============================================================================\n\ndef load_arff(file_path):\n    try:\n        data, _ = arff.loadarff(file_path)\n        df = pd.DataFrame(data)\n        for col in df.columns:\n            if df[col].dtype == object:\n                try: df[col] = df[col].str.decode('utf-8')\n                except: pass\n        return df\n    except:\n        with open(file_path, 'r', encoding='utf-8', errors='ignore') as f:\n            content = f.read()\n        data_start = content.lower().find('@data')\n        data_section = content[data_start + 5:].strip()\n        return pd.read_csv(StringIO(data_section), header=None)\n\ndef calc_metrics(y_true, y_pred, y_proba=None):\n    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()\n    m = {\n        'recall': recall_score(y_true, y_pred, zero_division=0),\n        'precision': precision_score(y_true, y_pred, zero_division=0),\n        'f1': f1_score(y_true, y_pred, zero_division=0),\n        'f2': fbeta_score(y_true, y_pred, beta=2, zero_division=0),\n        'accuracy': accuracy_score(y_true, y_pred),\n        'tp': int(tp), 'fp': int(fp), 'tn': int(tn), 'fn': int(fn)\n    }\n    if y_proba is not None:\n        try: m['pr_auc'] = average_precision_score(y_true, y_proba)\n        except: m['pr_auc'] = 0.0\n    else: m['pr_auc'] = 0.0\n    return m\n\ndef find_threshold(y_true, y_proba):\n    best_score, best_t = -1, 0.5\n    for t in np.arange(0.05, 0.96, 0.05):\n        y_pred = (y_proba >= t).astype(int)\n        m = calc_metrics(y_true, y_pred)\n        score = m['f2'] if m['accuracy'] >= 0.5 else 0\n        if score > best_score:\n            best_score, best_t = score, t\n    return best_t\n\n# ============================================================================\n# MODEL DEFINITIONS\n# ============================================================================\n\nclass KANLinear(nn.Module):\n    def __init__(self, in_f, out_f, grid=3, order=2):\n        super().__init__()\n        self.grid = nn.Parameter(torch.linspace(-1, 1, grid).unsqueeze(0).unsqueeze(0).repeat(out_f, in_f, 1))\n        self.coef = nn.Parameter(torch.randn(out_f, in_f, grid + order) * 0.1)\n        self.base = nn.Parameter(torch.randn(out_f, in_f) * 0.1)\n        self.in_f, self.out_f, self.grid_size, self.order = in_f, out_f, grid, order\n    \n    def forward(self, x):\n        bs = x.shape[0]\n        x_exp = x.unsqueeze(1).unsqueeze(-1)\n        dist = torch.abs(x_exp - self.grid.unsqueeze(0))\n        basis = torch.zeros(bs, self.out_f, self.in_f, self.grid_size + self.order, device=x.device)\n        for i in range(self.grid_size):\n            basis[:, :, :, i] = torch.exp(-dist[:, :, :, i] ** 2 / 0.5)\n        for i in range(self.order):\n            basis[:, :, :, self.grid_size + i] = x_exp.squeeze(-1) ** (i + 1)\n        spline = (basis * self.coef.unsqueeze(0)).sum(dim=-1).sum(dim=-1)\n        base = torch.matmul(x, self.base.t())\n        return spline + base\n\nclass KAN(nn.Module):\n    def __init__(self, in_dim, h=32, grid=3, order=2):\n        super().__init__()\n        self.kan1 = KANLinear(in_dim, h, grid, order)\n        self.kan2 = KANLinear(h, h // 2, grid, order)\n        self.out = nn.Linear(h // 2, 1)\n        self.bn1, self.bn2 = nn.BatchNorm1d(h), nn.BatchNorm1d(h // 2)\n        self.drop = nn.Dropout(0.3)\n    \n    def forward(self, x):\n        x = self.drop(F.relu(self.bn1(self.kan1(x))))\n        x = self.drop(F.relu(self.bn2(self.kan2(x))))\n        return torch.sigmoid(self.out(x))\n\nclass Attention(nn.Module):\n    def __init__(self, in_dim, att_dim=16):\n        super().__init__()\n        self.fc1 = nn.Linear(in_dim, att_dim)\n        self.fc2 = nn.Linear(att_dim, in_dim)\n        self.drop = nn.Dropout(0.2)\n    \n    def forward(self, x):\n        att = torch.sigmoid(self.fc2(self.drop(F.relu(self.fc1(x)))))\n        return x * att, att\n\nclass KAN_Att(nn.Module):\n    def __init__(self, in_dim, h=32, grid=3, order=2):\n        super().__init__()\n        self.att = Attention(in_dim, 16)\n        self.kan1 = KANLinear(in_dim, h, grid, order)\n        self.kan2 = KANLinear(h, h // 2, grid, order)\n        self.out = nn.Linear(h // 2, 1)\n        self.bn1, self.bn2 = nn.BatchNorm1d(h), nn.BatchNorm1d(h // 2)\n        self.drop = nn.Dropout(0.3)\n    \n    def forward(self, x):\n        x_att, _ = self.att(x)\n        x = self.drop(F.relu(self.bn1(self.kan1(x_att))))\n        x = self.drop(F.relu(self.bn2(self.kan2(x))))\n        return torch.sigmoid(self.out(x))\n\nclass FocalLoss(nn.Module):\n    def __init__(self, alpha=0.25, gamma=2.0):\n        super().__init__()\n        self.alpha, self.gamma = alpha, gamma\n    \n    def forward(self, inp, tgt):\n        bce = F.binary_cross_entropy(inp, tgt, reduction='none')\n        pt = torch.exp(-bce)\n        return (self.alpha * (1 - pt) ** self.gamma * bce).mean()\n\nprint('✅ All functions and models defined!')\n

## 🚀 Step 3: Complete Execution Pipeline

In [ ]:
# ============================================================================\n# COMPLETE EXECUTION FOR PC2\n# ============================================================================\n\nprint('\\n' + '='*70)\nprint('🚀 STARTING EXPERIMENT: PC2')\nprint('='*70 + '\\n')\n\n# Load data\nfile_path = os.path.join(DATASET_PATH, 'PC2.arff')\ndf = load_arff(file_path)\nX = df.iloc[:, :-1].values.astype(np.float32)\ny = df.iloc[:, -1].values\nif y.dtype == object:\n    y = LabelEncoder().fit_transform(y)\nelse:\n    y = y.astype(int)\n\n# Handle NaNs\nif np.any(np.isnan(X)):\n    col_med = np.nanmedian(X, axis=0)\n    inds = np.where(np.isnan(X))\n    X[inds] = np.take(col_med, inds[1])\n\nprint(f'📊 Loaded: {len(y)} samples, {X.shape[1]} features')\nprint(f'   Defective: {np.sum(y==1)} ({np.mean(y==1):.2%})\\n')\n\n# Split (leakage-free)\nX_train_full, X_test, y_train_full, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=SEED)\nX_train, X_val, y_train, y_val = train_test_split(X_train_full, y_train_full, test_size=0.2, stratify=y_train_full, random_state=SEED)\n\n# Scale\nscaler = MinMaxScaler()\nX_train = scaler.fit_transform(X_train)\nX_val = scaler.transform(X_val)\nX_test = scaler.transform(X_test)\n\nprint(f'🔀 Splits: Train={len(y_train)}, Val={len(y_val)}, Test={len(y_test)}\\n')\n\n# SMOTE (train only)\nsmote = SMOTE(sampling_strategy=0.7, random_state=SEED)\nX_train_sm, y_train_sm = smote.fit_resample(X_train, y_train)\nprint(f'🔄 SMOTE: {len(y_train)} → {len(y_train_sm)} (+{len(y_train_sm)-len(y_train)})\\n')\n\nresults = {}\n\n# ============================================================================\n# BASELINE: RANDOM FOREST\n# ============================================================================\nprint('🌲 Training Baseline RF...')\nrf = RandomForestClassifier(n_estimators=100, max_depth=10, class_weight='balanced', random_state=SEED, n_jobs=-1)\nrf.fit(X_train_sm, y_train_sm)\n\ny_val_proba_rf = rf.predict_proba(X_val)[:, 1]\nthresh_rf = find_threshold(y_val, y_val_proba_rf)\nprint(f'   Optimal threshold: {thresh_rf:.2f}')\n\ny_test_proba_rf = rf.predict_proba(X_test)[:, 1]\ny_test_pred_rf = (y_test_proba_rf >= thresh_rf).astype(int)\nmetrics_rf = calc_metrics(y_test, y_test_pred_rf, y_test_proba_rf)\n\nprint(f'   Test: Recall={metrics_rf[\"recall\"]:.4f}, Precision={metrics_rf[\"precision\"]:.4f}, F2={metrics_rf[\"f2\"]:.4f}\\n')\nresults['Baseline_RF'] = {'metrics': metrics_rf, 'threshold': thresh_rf}\n\n# ============================================================================\n# KAN BASE\n# ============================================================================\nprint('🔥 Training KAN Base...')\nmodel_kan = KAN(in_dim=X.shape[1], h=32, grid=3, order=2).to(device)\nopt = optim.Adam(model_kan.parameters(), lr=0.01)\ncriterion = FocalLoss()\n\nX_tr_t = torch.FloatTensor(X_train_sm).to(device)\ny_tr_t = torch.FloatTensor(y_train_sm).unsqueeze(1).to(device)\nX_val_t = torch.FloatTensor(X_val).to(device)\n\nds = TensorDataset(X_tr_t, y_tr_t)\nloader = DataLoader(ds, batch_size=64, shuffle=True)\n\nbest_f2, patience_cnt = 0, 0\nfor epoch in range(50):\n    model_kan.train()\n    for bx, by in loader:\n        opt.zero_grad()\n        loss = criterion(model_kan(bx), by)\n        loss.backward()\n        opt.step()\n    \n    model_kan.eval()\n    with torch.no_grad():\n        val_pred = (model_kan(X_val_t).cpu().numpy().flatten() >= 0.5).astype(int)\n        val_f2 = fbeta_score(y_val, val_pred, beta=2, zero_division=0)\n    \n    if val_f2 > best_f2:\n        best_f2, patience_cnt = val_f2, 0\n        best_state = model_kan.state_dict().copy()\n    else:\n        patience_cnt += 1\n    \n    if patience_cnt >= 10:\n        model_kan.load_state_dict(best_state)\n        break\n\nprint(f'   Training complete (best val F2: {best_f2:.4f})')\n\nmodel_kan.eval()\nwith torch.no_grad():\n    y_val_proba_kan = model_kan(X_val_t).cpu().numpy().flatten()\n    X_test_t = torch.FloatTensor(X_test).to(device)\n    y_test_proba_kan = model_kan(X_test_t).cpu().numpy().flatten()\n\nthresh_kan = find_threshold(y_val, y_val_proba_kan)\nprint(f'   Optimal threshold: {thresh_kan:.2f}')\n\ny_test_pred_kan = (y_test_proba_kan >= thresh_kan).astype(int)\nmetrics_kan = calc_metrics(y_test, y_test_pred_kan, y_test_proba_kan)\n\nprint(f'   Test: Recall={metrics_kan[\"recall\"]:.4f}, Precision={metrics_kan[\"precision\"]:.4f}, F2={metrics_kan[\"f2\"]:.4f}\\n')\nresults['KAN_Base'] = {'metrics': metrics_kan, 'threshold': thresh_kan}\n\n# ============================================================================\n# KAN + ATTENTION\n# ============================================================================\nprint('🌟 Training KAN + Attention...')\nmodel_att = KAN_Att(in_dim=X.shape[1], h=32, grid=3, order=2).to(device)\nopt = optim.Adam(model_att.parameters(), lr=0.01)\ncriterion = FocalLoss()\n\nbest_f2, patience_cnt = 0, 0\nfor epoch in range(50):\n    model_att.train()\n    for bx, by in loader:\n        opt.zero_grad()\n        loss = criterion(model_att(bx), by)\n        loss.backward()\n        opt.step()\n    \n    model_att.eval()\n    with torch.no_grad():\n        val_pred = (model_att(X_val_t).cpu().numpy().flatten() >= 0.5).astype(int)\n        val_f2 = fbeta_score(y_val, val_pred, beta=2, zero_division=0)\n    \n    if val_f2 > best_f2:\n        best_f2, patience_cnt = val_f2, 0\n        best_state = model_att.state_dict().copy()\n    else:\n        patience_cnt += 1\n    \n    if patience_cnt >= 10:\n        model_att.load_state_dict(best_state)\n        break\n\nprint(f'   Training complete (best val F2: {best_f2:.4f})')\n\nmodel_att.eval()\nwith torch.no_grad():\n    y_val_proba_att = model_att(X_val_t).cpu().numpy().flatten()\n    y_test_proba_att = model_att(X_test_t).cpu().numpy().flatten()\n\nthresh_att = find_threshold(y_val, y_val_proba_att)\nprint(f'   Optimal threshold: {thresh_att:.2f}')\n\ny_test_pred_att = (y_test_proba_att >= thresh_att).astype(int)\nmetrics_att = calc_metrics(y_test, y_test_pred_att, y_test_proba_att)\n\nprint(f'   Test: Recall={metrics_att[\"recall\"]:.4f}, Precision={metrics_att[\"precision\"]:.4f}, F2={metrics_att[\"f2\"]:.4f}\\n')\nresults['KAN_Attention'] = {'metrics': metrics_att, 'threshold': thresh_att}\n\n# ============================================================================\n# SUMMARY & EXPORT\n# ============================================================================\nprint('\\n' + '='*70)\nprint(f'📊 FINAL RESULTS - PC2')\nprint('='*70 + '\\n')\n\nres_list = []\nfor model_name, data in results.items():\n    m = data['metrics']\n    res_list.append({\n        'dataset': 'PC2',\n        'model': model_name,\n        'recall': m['recall'],\n        'precision': m['precision'],\n        'f1': m['f1'],\n        'f2': m['f2'],\n        'accuracy': m['accuracy'],\n        'pr_auc': m['pr_auc'],\n        'threshold': data['threshold']\n    })\n\nresults_df = pd.DataFrame(res_list)\n\nfor _, row in results_df.iterrows():\n    print(f\"{row['model']}:\")\n    print(f\"   Recall:    {row['recall']:.4f} {'🎯' if row['recall'] >= 0.80 else ''}\")\n    print(f\"   Precision: {row['precision']:.4f}\")\n    print(f\"   F2:        {row['f2']:.4f}\")\n    print(f\"   Accuracy:  {row['accuracy']:.4f}\")\n    print(f\"   Threshold: {row['threshold']:.2f}\\n\")\n\n# Export\ncsv_path = os.path.join(OUTPUT_DIR, f'results_{RUN_ID}.csv')\njson_path = os.path.join(OUTPUT_DIR, f'results_{RUN_ID}.json')\n\nresults_df.to_csv(csv_path, index=False)\nresults_df.to_json(json_path, orient='records', indent=2)\n\nprint(f'\\n💾 Results saved:')\nprint(f'   CSV:  {csv_path}')\nprint(f'   JSON: {json_path}')\n\nprint('\\n' + '='*70)\nprint(f'✅ EXPERIMENT COMPLETE - PC2')\nprint('='*70)\n